# 特征重要性 (Feature Importance)

## 概述

特征重要性是评估每个特征对模型预测贡献程度的指标。树模型提供了多种计算特征重要性的方法：

1. **基于不纯度减少 (MDI)**: 计算特征在所有树中带来的不纯度减少量
2. **基于排列重要性 (Permutation)**: 打乱特征值后观察模型性能下降程度

## 不纯度减少的计算

对于每个特征，计算其在所有树的所有节点上的加权不纯度减少：

$$\text{Importance}(f) = \sum_{\text{nodes using } f} \frac{N_t}{N} \cdot \Delta I$$

其中 $N_t$ 是节点的样本数，$\Delta I$ 是不纯度减少量。

In [ ]:
# 导入必要的库
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. 数据准备

In [ ]:
# 加载鸢尾花数据集
iris = load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"特征名称: {feature_names}")
print(f"特征数量: {len(feature_names)}")

## 2. 随机森林特征重要性

随机森林的特征重要性基于平均不纯度减少（Mean Decrease in Impurity, MDI）。

In [ ]:
# 训练随机森林
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_leaf_nodes=16,
    random_state=RANDOM_STATE
)
rf_clf.fit(X_train, y_train)

# 获取特征重要性
rf_importances = rf_clf.feature_importances_

# 按重要性排序
indices = np.argsort(rf_importances)[::-1]

print("随机森林特征重要性 (MDI):")
print("-" * 40)
for i in indices:
    print(f"  {feature_names[i]:<20}: {rf_importances[i]:.4f}")

## 3. 梯度提升特征重要性

In [ ]:
# 训练梯度提升分类器
gb_clf = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=3,
    random_state=RANDOM_STATE
)
gb_clf.fit(X_train, y_train)

# 获取特征重要性
gb_importances = gb_clf.feature_importances_

# 按重要性排序
indices_gb = np.argsort(gb_importances)[::-1]

print("梯度提升特征重要性 (MDI):")
print("-" * 40)
for i in indices_gb:
    print(f"  {feature_names[i]:<20}: {gb_importances[i]:.4f}")

## 4. 排列重要性 (Permutation Importance)

排列重要性通过打乱特征值来评估特征的重要性，不依赖于模型内部结构。

In [ ]:
# 计算排列重要性
perm_importance = permutation_importance(
    rf_clf, X_test, y_test,
    n_repeats=10,
    random_state=RANDOM_STATE
)

# 排序
perm_indices = np.argsort(perm_importance.importances_mean)[::-1]

print("排列重要性 (Permutation Importance):")
print("-" * 50)
for i in perm_indices:
    mean = perm_importance.importances_mean[i]
    std = perm_importance.importances_std[i]
    print(f"  {feature_names[i]:<20}: {mean:.4f} (+/- {std:.4f})")

## 5. 不同方法对比

In [ ]:
# 对比三种方法
print("特征重要性对比:")
print(f"{'特征':<20} {'RF-MDI':>10} {'GB-MDI':>10} {'Permutation':>12}")
print("-" * 54)

for i in range(len(feature_names)):
    print(f"{feature_names[i]:<20} {rf_importances[i]:>10.4f} "
          f"{gb_importances[i]:>10.4f} "
          f"{perm_importance.importances_mean[i]:>12.4f}")

## 6. 单元测试验证

In [ ]:
def test_feature_importance():
    """特征重要性功能测试"""
    
    # 测试1: 特征重要性维度应该正确
    assert len(rf_importances) == len(feature_names), "RF重要性维度不正确"
    assert len(gb_importances) == len(feature_names), "GB重要性维度不正确"
    
    # 测试2: 特征重要性之和应该接近1
    assert abs(sum(rf_importances) - 1.0) < 0.01, "RF重要性之和不正确"
    assert abs(sum(gb_importances) - 1.0) < 0.01, "GB重要性之和不正确"
    
    # 测试3: 所有重要性值应该非负
    assert all(imp >= 0 for imp in rf_importances), "RF存在负重要性"
    assert all(imp >= 0 for imp in gb_importances), "GB存在负重要性"
    
    # 测试4: 排列重要性应该有正确的形状
    assert perm_importance.importances.shape[0] == len(feature_names), "排列重要性维度不正确"
    
    # 测试5: 花瓣特征应该比萼片特征更重要（鸢尾花数据集的已知特性）
    petal_indices = [2, 3]  # petal length, petal width
    sepal_indices = [0, 1]  # sepal length, sepal width
    petal_importance = sum(rf_importances[i] for i in petal_indices)
    sepal_importance = sum(rf_importances[i] for i in sepal_indices)
    assert petal_importance > sepal_importance, "花瓣特征应该更重要"
    
    print("所有测试通过!")

test_feature_importance()

## 总结

### MDI vs Permutation Importance

| 特性 | MDI | Permutation |
|------|-----|-------------|
| 计算速度 | 快（训练时计算） | 慢（需额外计算） |
| 高基数偏差 | 存在偏差 | 无偏差 |
| 相关特征 | 可能分散重要性 | 同样受影响 |
| 模型依赖 | 仅限树模型 | 适用任何模型 |

### 使用建议

1. **快速评估**: 使用 MDI（`feature_importances_`）
2. **可靠评估**: 使用排列重要性（`permutation_importance`）
3. **特征选择**: 结合两种方法的结果

### 注意事项

- MDI 倾向于高估高基数特征的重要性
- 相关特征可能会分散重要性
- 在测试集上计算排列重要性更可靠